In [2]:
import requests
import json
import re
import pandas as pd
from datetime import datetime

In [12]:
url = (
    "https://stock2.finance.sina.com.cn/futures/api/jsonp.php"
    "?symbol=T2112&type=15=/InnerFuturesNewService.getFewMinLine"
)
res = requests.get(url)
df = (
    pd.DataFrame(json.loads(re.findall(r"\[.*?\]", res.text)[0]))
    .rename(
        columns={
            "d": "datetime",
            "o": "open",
            "h": "high",
            "l": "low",
            "c": "close",
            "v": "volume",
            "p": "position",
        }
    )
    .set_index("datetime")
    .rename(index=lambda idx: pd.to_datetime(idx))
    .astype(
        {
            **{col_name: "float64" for col_name in ["open", "high", "low", "close"]},
            **{col_name: "int64" for col_name in ["volume", "position"]},
        }
    )
)

# 如果现在的时间在 df 最后一行的时间之前
# 就抛弃最后一行
if (datetime.now() - df.index[-1]).total_seconds() < 0:
    df = df.head(-1)


In [13]:
df.loc['2021-09-02']

,open,high,low,close,volume,position
datetime,,,,,,
2021-09-02 09:45:00,100.360,100.375,100.320,100.340,8561,151714
2021-09-02 10:00:00,100.335,100.395,100.305,100.385,5336,152328
2021-09-02 10:15:00,100.380,100.385,100.350,100.365,2450,152574
2021-09-02 10:30:00,100.370,100.430,100.365,100.375,3677,152801
2021-09-02 10:45:00,100.375,100.410,100.370,100.380,1870,152816
2021-09-02 11:00:00,100.380,100.390,100.355,100.370,1481,152825
2021-09-02 11:15:00,100.370,100.390,100.345,100.355,1990,152662
2021-09-02 11:30:00,100.360,100.395,100.355,100.390,1511,152435
2021-09-02 13:15:00,100.390,100.455,100.360,100.440,2575,153100


In [14]:
df.dtypes

open        float64
high        float64
low         float64
close       float64
volume        int64
position      int64
dtype: object